In [2]:
%load_ext autoreload
%autoreload 2

In [8]:
from kfsims.common import init_all, init_trajectory

In [12]:
#from trajectory import trajectory
from vbkf.vb_update import *

#traj = trajectory(ndat=300)
#traj.R = 5 * np.eye(2)
#traj._simulate()
traj = init_trajectory(300)


xk = np.array([0, 0, 1, 1])
P = 100 * np.eye(4)
tau = 5
n = 4
rho = .99

u = 13
m = 2
U = 2*traj.R * (u - m - 1)

H = traj.H
F = traj.A
Q = 2 * traj.Q
N = 20

In [13]:
t = 0
x_log = []
P_log = []
for zk in traj.Y.T:
#    print('===== Step: ', t, '======') 
    t += 1
    
    #### Time update
    xk = predict_state(F, xk)
    P = predict_PECM(F, P, Q)
    
    #### Measurement update
    # Initialization - step 3
    tkk, Tkk, ukk, Ukk = init(P, tau, n, rho, u, m, U)
    
    ## VB iters
    Pik = P
    xikk = xk
    for i in range(N):
        # steps 4, 5
        err = np.atleast_2d(xikk - xk)
        Aik = Pik + np.outer(err, err)
        tik = tkk + 1
        Tik = Aik + Tkk
        
        # steps 6, 7
        err = np.atleast_2d(zk - H.dot(xikk))
        Bik = err.T.dot(err) + H.dot(Pik).dot(H.T)
        uik = ukk + 1
        Uik = Bik + Ukk
        
        # steps 8
        ERkinv = (uik - m - 1) * np.linalg.inv(Uik)
        EPkinv = (tik - n - 1) * np.linalg.inv(Tik)
        
        # steps 9
        Pik = np.linalg.inv(EPkinv)
        Rik = np.linalg.inv(ERkinv)
        
        # steps 10-12
        bracket = H.dot(Pik).dot(H.T) + Rik
        Kik = Pik.dot(H.T).dot(np.linalg.inv(bracket))
        xikk = xk + Kik.dot(zk - H.dot(xk))
        Pik = Pik - Kik.dot(H).dot(Pik)
    
    xk = xikk
    P = Pik
    Tkk = Tik
    ukk = uik
    Ukk = Uik
    
    x_log.append(xk)
    P_log.append(P)

x_log = np.array(x_log).squeeze().T
P_log = np.array(P_log).squeeze()

print(np.sqrt(((x_log - traj.X)**2).mean(axis=1)))


[1.18015922 0.9639272  1.45376622 1.37816697]
